In [6]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env

api_key = os.environ.get("OPENAI_API_KEY")
print(f"API Key Loaded: {bool(api_key)}")  # Optional check

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")


API Key Loaded: True


In [7]:
from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner

agent = Agent(
    name="Joker",
    instructions="You are a helpful assistant.",
)

result = Runner.run_streamed(agent, input="Please fix my daily routine, give 5 tips")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

I'd be happy to help! Here are five general tips to improve your daily routine:

1. **Prioritize Sleep:**
   Aim for 7-9 hours of quality sleep each night. Create a relaxing bedtime routine and maintain a consistent sleep schedule, even on weekends. This helps regulate your body's internal clock.

2. **Morning Mindfulness:**
   Start your day with a moment of mindfulness or meditation. Even just 5-10 minutes of focused breathing or gratitude journaling can set a positive tone for the day.

3. **Plan and Organize:**
   Spend a few minutes in the morning (or the night before) to plan your day. Use a planner or digital tool to prioritize tasks and allocate time effectively. Break tasks into manageable chunks.

4. **Incorporate Physical Activity:**
   Include some form of exercise in your daily routine. This could be a morning jog, a lunchtime walk, or evening yoga. Regular physical activity boosts energy levels and improves mood.

5. **Balanced Nutrition:**
   Pay attention to what you ea

In [ ]:
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)

agent = Agent(
    name="Joker",
    instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
    tools=[how_many_jokes],
)

result = Runner.run_streamed(
    agent,
    input="Hello",
)
print("=== Run starting ===")

async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 6
-- Message output:
 Here are six jokes for you:

1. **Why don't scientists trust atoms?**
   Because they make up everything!

2. **Did you hear about the mathematician who’s afraid of negative numbers?**
   He'll stop at nothing to avoid them.

3. **Why was the math book sad?**
   Because it had too many problems.

4. **What do you call fake spaghetti?**
   An impasta!

5. **Why did the scarecrow win an award?**
   Because he was outstanding in his field!

6. **Have you heard about the restaurant on the moon?**
   Great food, no atmosphere.
=== Run complete ===
